# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

## NetCDF configuration package and visualization
  - The `NCF6` package is a subpackage of the model discretization (`DIS` or `DISV`) package
  - Additional configuration options are available in the `NCF6` package related to NetCDF compression and visualization
  - These include a `WKT` (well-known text) string that can aid visualization in applications like QGIS or Panoply

In [ ]:
import pathlib as pl

import flopy
import numpy as np
import xarray as xa
import xugrid as xu

### Load the modflow6-examples base advtidal model

In [ ]:
name = "advtidal"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("data/ex-gwf-advtidal/")
)

### Modify the model to save NetCDF output
  - Set `start_date_time` in the TDIS file

In [ ]:
sim.tdis.start_date_time = "1980-01-01"

  - Define `export_netcdf` in the groundwater flow model `name_file`, specifying `structured` as the output type

In [ ]:
gwf = sim.get_model()
gwf.export_netcdf = "structured"

### Define `WKT` string and create `NCF` package
  - This is a synthetic model- no actual georeference data exists.
  - For demonstration, assign wkt string for EPSG 5070 (NAD83 / Conus Albers)

In [ ]:
wkt = """PROJCRS["NAD83 / Conus Albers",BASEGEOGCRS["NAD83",DATUM["North American Datum 1983",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4269]],CONVERSION["Conus Albers",METHOD["Albers Equal Area",ID["EPSG",9822]],PARAMETER["Latitude of false origin",23,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",-96,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude of 1st standard parallel",29.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",45.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Easting at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8826]],PARAMETER["Northing at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8827]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["Data analysis and small scale data presentation for contiguous lower 48 states."],AREA["United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming."],BBOX[24.41,-124.79,49.38,-66.91]],ID["EPSG",5070]]"""

  - Assign `XORIGIN` and `YORIGIN` to locate grid in contiguous US

In [ ]:
gwf.dis.xorigin = -1660000.0
gwf.dis.yorigin = 2440000.0

In [ ]:
ncf = flopy.mf6.ModflowUtlncf(
    gwf.dis,
    ogc_wkt=wkt,
    filename="ex-gwf-advtidal.dis.ncf",
)

### Run the model
  - Change simulation workspace to structured subdirectory

In [ ]:
ws = pl.Path("temp/step2/advtidal/structured")
sim.set_sim_path(ws)

  - Write model files and run the model

In [ ]:
sim.write_simulation()
sim.run_simulation()

### Rename NetCDF export
  - For clarity in post-processing step, rename export

In [ ]:
nc_fpth = pl.Path("temp/step2/advtidal/structured/ex-gwf-advtidal.nc")
nc_fpth.rename("temp/step2/advtidal/structured/ex-gwf-advtidal.structured.nc")

### Modify the model to export mesh NetCDF file

In [ ]:
gwf = sim.get_model()
gwf.export_netcdf = "ugrid"

### Run the model
  - Change simulation workspace to mesh subdirectory

In [ ]:
ws = pl.Path("temp/step2/advtidal/mesh")
sim.set_sim_path(ws)

  - Write model files and run the model

In [ ]:
sim.write_simulation()
sim.run_simulation()

### Rename NetCDF export
  - For clarity in post-processing step, rename export

In [ ]:
nc_fpth = pl.Path("temp/step2/advtidal/mesh/ex-gwf-advtidal.nc")
nc_fpth.rename("temp/step2/advtidal/mesh/ex-gwf-advtidal.mesh.nc")

### Load the modflow6-examples base GWF synthetic valley model

In [ ]:
name = "synthetic-valley"
gwf_sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("data/ex-gwt-synthetic-valley/mf6gwf")
)

### Load the modflow6-examples base GWT synthetic valley model

In [ ]:
gwt_sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("data/ex-gwt-synthetic-valley/mf6gwt")
)

### Modify the GWT model to save UGRID NetCDF output
  - Set `start_date_time` in the TDIS file 

In [ ]:
gwt_sim.tdis.start_date_time = "1980-01-01"

  - Define `export_netcdf` in the groundwater transport model `name_file`, specifying `ugrid` as the output type

In [ ]:
gwt = gwt_sim.get_model("trans")
gwt.export_netcdf = "ugrid"

### Define `WKT` string and create `NCF` package
  - This is a synthetic model- no actual georeference data exists.
  - For demonstration, assign wkt string for EPSG 5070 (NAD83 / Conus Albers)

In [ ]:
wkt = """PROJCRS["NAD83 / Conus Albers",BASEGEOGCRS["NAD83",DATUM["North American Datum 1983",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4269]],CONVERSION["Conus Albers",METHOD["Albers Equal Area",ID["EPSG",9822]],PARAMETER["Latitude of false origin",23,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",-96,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude of 1st standard parallel",29.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",45.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Easting at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8826]],PARAMETER["Northing at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8827]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["Data analysis and small scale data presentation for contiguous lower 48 states."],AREA["United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming."],BBOX[24.41,-124.79,49.38,-66.91]],ID["EPSG",5070]]"""

  - Assign `XORIGIN` and `YORIGIN` to locate grid in contiguous US

In [ ]:
gwt.disv.xorigin = -1620000.0
gwt.disv.yorigin = 2520000.0

  - Create the NCF6 package

In [ ]:
ncf = flopy.mf6.ModflowUtlncf(
    gwt.disv,
    ogc_wkt=wkt,
    filename="trans.disv.ncf",
)

### Run the model
  - Change simulation workspace to structured subdirectory
  - Write model files and run the model

### Modfiy the model to export mesh NetCDF file
  - Specify `UGRID` as output type

In [ ]:
gwt = gwt_sim.get_model("trans")
gwt.export_netcdf = "ugrid"

  - Export `DISV` griddata input arrays (for visualization)

In [ ]:
gwt.disv.export_array_netcdf = True

### Run the GWF model
  - Change simulation workspace to mesh subdirectory

In [ ]:
gwf_ws = pl.Path("temp/step2/synthetic-valley/mf6gwf")
gwf_sim.set_sim_path(gwf_ws)

  - Write model files and run the GWF model

In [ ]:
gwf_sim.write_simulation()
gwf_sim.run_simulation()

### Run the GWT model
  - Change simulation workspace to mesh subdirectory

In [ ]:
gwt_ws = pl.Path("temp/step2/synthetic-valley/mf6gwt")
gwt_sim.set_sim_path(gwt_ws)

  - Write model files and run the GWT model

In [ ]:
gwt_sim.write_simulation()
gwt_sim.run_simulation()

### Rename NetCDF export
  - For clarity in post-processing step, rename export

In [ ]:
nc_fpth = pl.Path("temp/step2/synthetic-valley/mf6gwt/trans.nc")
nc_fpth.rename("temp/step2/synthetic-valley/mf6gwt/trans.mesh.nc")